# Song Syllable Sequence Analysis

This notebook demonstrates the syllable network analysis functionality.

In [ ]:
import sys
sys.path.append('../src')

import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from numpy import random
from pathlib import Path
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from syllable_network_analysis.analysis import (
    get_trans_matrix,
    get_syllable_network,
    get_trans_entropy,
    get_sequence_linearity,
    get_sequence_consistency,
    get_song_stereotypy
)
from syllable_network_analysis.plot import plot_transition_diag

# Configure matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

## Load Sample Data

Sample data from one recording session:

In [ ]:
# Sample data from one recording session
syllables = 'kiiiiabcdjiabcdjiabcd*iiiabcdk*iiii*iiiabcdjiabcdk*kiiiiiabcdjia*kiiiiiabcdjjabcd*iiiiabcd*iiiiabcdk*iiiiab*k*iiiiabcdk*iiiabcdjiabcd*iiiabcd*iiiabcdjiabcdk*iiabcd*iiiiiabcdjiabcd*iiiiabcd*iiiiiabcdjiabcdk*k*iiiabcdjiak*iiiabcdjiaj*kmmiiiabcdjiabcd*iiiiabcdjiabcd*iiiiiabcdk*iiiiabcdjiabcd*iiiiabcd*'
song_note = ('a', 'b', 'c', 'd')
calls = ('m')
intro_notes = ('i')
note_seq = ('i', 'a', 'b', 'c', 'd', 'j', 'k', 'm', '*')  # syllabic sequence, asterisk indicates stop
song_name = '017-g35r38-Predeafening-D36-20190617'
cmap_list = ["YlGnBu", "PiYG"]

## Count Notes

Count the number of notes (except for stop):

In [ ]:
# Count the number of notes (except for stop)
note_count = {}

for note in set(syllables):
    note_count[note] = syllables.count(note)
note_count.pop('*')
note_count

## Audio Files

List available audio files:

In [ ]:
# List available audio files
data_dir = Path("../data/raw/")
audio_files = list(data_dir.glob('*.wav'))
for file in audio_files:
    print(file.name)

## Assign Colors to Syllables

Assign different colors for different syllable types:

In [ ]:
# Assign colors to syllables
motif_color = ['r', 'b', 'lime', 'm', 'darkorange', 'purple', 'saddlebrown']
intro_color = ['k', 'gray', 'darkseagreen', 'olive'] 
call_color = ['teal', 'darkslategray', 'darkgray','indigo']

song_note = ['a', 'b', 'c', 'd']
calls = ['m']
intro_notes = ['i']

syl_color = []

for i, note in enumerate(note_seq[:-1]):
    if note in song_note:
        syl_color.append(motif_color.pop(0))
    elif note in intro_notes:
        syl_color.append(intro_color.pop(0))
    elif note in calls:
        syl_color.append(call_color.pop(0))
    else:
        syl_color.append(intro_color.pop(0))
        
syl_color.append('y')  # for stop at the end 

print(note_seq)
print(syl_color)

## Analyze Syllable Transitions

Build transition matrix and analyze syllable network:

In [ ]:
# Build transition matrix
trans_matrix = get_trans_matrix(syllables, note_seq)
print("Transition Matrix:")
print(trans_matrix)

# Get syllable network
syl_network = get_syllable_network(trans_matrix)
print(f"\nSyllable Network ({len(syl_network)} transitions):")
for start, end, weight in syl_network:
    print(f"  {note_seq[start]} -> {note_seq[end]}: {weight}")

## Calculate Metrics

Calculate various sequence analysis metrics:

In [ ]:
# Calculate metrics
trans_entropy = get_trans_entropy(trans_matrix)
sequence_linearity = get_sequence_linearity(note_seq, syl_network)
sequence_consistency = get_sequence_consistency(note_seq, trans_matrix)
song_stereotypy = get_song_stereotypy(sequence_linearity, sequence_consistency)

print(f"Transition Entropy: {trans_entropy:.4f}")
print(f"Sequence Linearity: {sequence_linearity:.4f}")
print(f"Sequence Consistency: {sequence_consistency:.4f}")
print(f"Song Stereotypy: {song_stereotypy:.4f}")

## Visualize Transition Diagram

Create a visualization of the syllable transition network:

In [ ]:
# Create color mapping for visualization
syl_color_dict = dict(zip(note_seq, syl_color))

# Create the plot
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
plot_transition_diag(ax, note_seq, syl_network, syl_color_dict)
plt.title(f"Syllable Transition Network - {song_name}", fontsize=16)
plt.tight_layout()
plt.show()